In [ ]:
#dependencies
!pip install kneed
# Load the required modules
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import scvi
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA
import warnings
from scipy.stats import median_abs_deviation
import os
import pickle as pkl
from kneed import KneeLocator as kl
import scrublet as scr
import os
# Ignore the warning messages
warnings.filterwarnings("ignore")

In [ ]:
import scanpy.external as sce

In [ ]:
!ls

In [ ]:
# Path of folder
ruta_carpeta = "./adatas"

# List to store all the AnnData objects
adatas = []

# Go through all the files in the folder 
for archivo in os.listdir(ruta_carpeta):
    ruta_archivo = os.path.join(ruta_carpeta, archivo)
    # Verify if the file is valid for AnnData (e.g., .h5ad, .loom, .mtx, etc.)
    if os.path.isfile(ruta_archivo) and archivo.endswith((".h5ad", ".loom", ".mtx")):
        try:
            # Load the file as AnnData
            adata = sc.read_h5ad(ruta_archivo)
            adata.var_names_make_unique()
            adata.obs_names_make_unique()
            adatas.append(adata)
            print(f"Loaded: {archivo}")
        except Exception as e:
            print(f"Error when loading {archivo}: {e}")

# Result: list of AnnData objects
print(f"Loaded {len(adatas)} AnnData files.")


In [ ]:
gene_sets = [set(adata.var_names) for adata in adatas]

# Find the intersection of genes between adatas
shared_genes = set.intersection(*gene_sets)

# Result: number of shared genes
print(f"Number of shared genes: {len(shared_genes)}")

In [ ]:
adata = sc.concat(adatas)

In [ ]:
adata.obs

In [ ]:
adata.write("adata_raw.h5ad")

In [ ]:
adata = sc.read_h5ad("adata_raw.h5ad")

In [ ]:
adata.X.max(), adata.X.min()

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:
#Calculate the QC covariates or metric

# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
#Calculate the respective QC metrics 
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)
adata

In [ ]:
sns.displot(adata.obs["total_counts"], bins=100, kde=False)
# sc.pl.violin(adata, 'total_counts')
sc.pl.violin(adata, "pct_counts_mt")
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

In [ ]:
adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (adata.obs["pct_counts_mt"] > 40)

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

In [ ]:
p1 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
adata.layers['counts'] = csr_matrix(adata.X) # Save the non normalized data in a compressed matrix
    
adata.raw = adata # Save the non normalized data counts in the raw atribute
sc.pp.normalize_total(adata, target_sum=1e4) 
sc.pp.log1p(adata)

In [ ]:
# Select and compute highly variable genes 
sc.pp.highly_variable_genes(adata,n_top_genes=2000)

# Plot variable genes
sc.pl.highly_variable_genes(adata)

# Actually do the filtering and subset for variable genes in the dataset
adata = adata[:, adata.var.highly_variable].copy()

adata.layers['hv_counts'] = adata.X.copy() # Save the normalized highly variable counts in a new layer

In [ ]:
# Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# Define new function that finds the elbow dimension 
def PCA_Elbow_fit(data):
    model = PCA().fit(data)
    explained_variance = model.explained_variance_ratio_
    pcs = list(range(1, explained_variance.shape[0]+1))#enumerate(explained_variance,1)
    klm = kl(pcs, explained_variance, S=1.0, curve='convex', direction='decreasing')
    pcs_used = klm.knee
    pc_list = list(range(1, pcs_used+1))
    new_data = PCA(n_components= pcs_used, svd_solver='arpack').fit_transform(data)

    if pcs_used >50:
        pcs_used=50
        
    return pcs_used, new_data, pcs, explained_variance, pc_list

In [ ]:
# Extract cell x gene data from scanpy annData and create new pandas dataframe

new_frame = pd.DataFrame(adata.X, index=adata.obs_names, columns=adata.var_names)

pandas_data = new_frame.values

# Execute PCA elbow_fitting_funcion
dim,new_matrix,pc_ax,pc_ay,col_labels=PCA_Elbow_fit(pandas_data)
print(dim)

# Neigbours computation and umap
sc.pp.neighbors(adata, n_pcs = dim) # compute nearest neighbors
sc.tl.umap(adata)

# Plot UMAP
sc.pl.umap(adata, color = ['Sample'], frameon = False,title="NON INTEGRATED SAMPLES")

In [ ]:
sc.pl.umap(adata, color = ['Sample'], frameon = False,title="NON INTEGRATED SAMPLES")

### INTEGRACION

In [ ]:
sce.pp.harmony_integrate(adata, 'Sample')

In [ ]:
# Compute neighbors using scVI model
sc.pp.neighbors(adata, use_rep = 'X_pca_harmony', metric="correlation")


In [ ]:
# Generate UMAP and leiden clustering
sc.tl.umap(adata, min_dist=0.4)
sc.tl.leiden(adata, resolution = 0.7) #key_added="leiden_scvi"

print(f"Plotting integrated data UMAP:")

sc.pl.umap(adata, color = ['Sample'], frameon = False,title="INTEGRATED SAMPLES")
sc.pl.umap(adata, color = ['leiden'],legend_loc = 'on data', frameon = False,title="LEIDEN CLUSTERING")

In [ ]:
# Define a function that gives a new label to the cells of a cluster (for manual annotation)

def relabel_cluster_celltypes(adata,new_label,target_clusters,cluster_key="leiden",cell_type_key="cell_type",inplace=True,new_column_key= None):

    values_to_change = adata.obs[cluster_key].isin(target_clusters)

    celltypeslist = adata.obs[cell_type_key].to_numpy()

    celltypeslist[values_to_change] = new_label

    if inplace:

        adata.obs[cell_type_key] = celltypeslist

    else:
        adata.obs[new_column_key] = celltypeslist

In [ ]:
adata.obs['cell_type'] = 'Unknown'

In [ ]:
curated_dict = {"Tumor cells": ['EPCAM', "CDH1", "KRT19", "KRT7", "SOX9"],
                "Fibroblasts": ["DCN","FBLN1","LUM"],
                "T CD4": ["CD4","GATA3","FOXP3","IL7R"],
                "T CD8": ["GZMB","CD8A","PRF1"],
                "B cell" : ['VPREB3', 'CD19', 'MS4A1'],
                "Stellate cells": ["PDGFRB"],
                "Endothelial cells":["VWF","EGFL7","EMCN"],
                "Macrophages": ["CD68","CD14","CD163"],
                "Dendritic cells": ["CD1C"],
                "Acinar cells": ["KLK1","PRSS3","CELA2B"],
                "Alpha cells":["GCG","PAX6","CHGB"],
                "Plasma cells":["MZB1","TNFRSF17","FKBP11","SSR4"],
                "Mast cells":["KIT", "TPSAB1","CPA3"],
                "my-CAF":["FAP","PDPN","ACTA2"]
                }

## Tumor cells

In [ ]:
sc.pl.umap(adata,color=curated_dict["Tumor cells"],frameon=False,use_raw=True, vmax= 20)

In [ ]:
relabel_cluster_celltypes(adata,new_label="Tumor cells",target_clusters=["3","13","16","19"])

## Fibroblasts

In [ ]:
sc.pl.umap(adata,color=curated_dict["Fibroblasts"],frameon=False, vmax= 100)

In [ ]:
sc.pl.umap(adata,color=curated_dict["my-CAF"],frameon=False,use_raw=True, vmax = 100)

In [ ]:
relabel_cluster_celltypes(adata,new_label="Fibroblasts",target_clusters=["8"])

## Endothelial cells

In [ ]:
sc.pl.umap(adata,color=curated_dict["Endothelial cells"],frameon=False,use_raw=True, vmax = 30)

In [ ]:
relabel_cluster_celltypes(adata,new_label="Endothelial Cells", target_clusters=["14"])

## B cells

In [ ]:
sc.pl.umap(adata,color=curated_dict["B cell"],frameon=False,use_raw=True, vmax= 10)

In [ ]:
relabel_cluster_celltypes(adata,new_label="B cells",target_clusters=["7"])

### T/NK Cells

In [ ]:
#General T cells
sc.pl.umap(adata, color=["CD3D", "CD3E", "CD3G", "CD5", "CD7", "CD8A", "CD4", "CD28"], vmax= 10)

In [ ]:
#T CD4 Helper
sc.pl.umap(adata, color=["CD4", "CD3D", "IL7R", "FOXP3", "CTLA4", "CXCR5", "GATA3"], vmax=20)

In [ ]:
#T CD8 Cytotoxic
sc.pl.umap(adata, color=["CD8A", "CD8B", "GZMK", "PRF1", "GZMA", "IFNG", "CD69", "NKG7"], vmax = 50)

In [ ]:
#NK
sc.pl.umap(adata, color=["NKG7", "KLRD1", "KLRB1", "NCAM1", "GZMB", "GZMK", "IFNG"], vmax = 50)

In [ ]:
#Tregs
sc.pl.umap(adata, color=["FOXP3", "CD4", "CTLA4", "IL10"], vmax = 20)

In [ ]:
relabel_cluster_celltypes(adata,new_label="T cells",target_clusters=["0", "1", "2", "6", "9", "11", "15"])

## Myeloid cells: (neutrophils, macrophages, eosinophils, basophils, monocytes and dendritic cells)

In [ ]:
#Neutrophils
sc.pl.umap(adata, color=['CEACAM1', 'ELANE', 'MPO', 'S100A8', 'S100A9'], vmax = 100)

In [ ]:
#Macrophages
sc.pl.umap(adata, color=['CD68', 'CD163', 'CSF1R', 'ITGAM', 'TLR4'], vmax = 50)

In [ ]:
#Eosinophils
sc.pl.umap(adata, color=['IL5RA', 'EGF', 'CCR3'], vmax= 50)

In [ ]:
#Basophils
sc.pl.umap(adata, color=['KIT', 'FCER1A'], vmax= 20)

In [ ]:
#Monocytes
sc.pl.umap(adata, color=['CD14', 'CSF1R', 'CCR2'], vmax = 40)

In [ ]:
#Dendritic cells
sc.pl.umap(adata, color=['CD80', 'CD86', 'CLEC9A', 'IRF8'], vmax = 20)

In [ ]:
relabel_cluster_celltypes(adata,new_label="Myeloid Cells ",target_clusters=["4","5","10","12","18"])

## Mast cells

In [ ]:
sc.pl.umap(adata, color=["TPSAB1", "TPSB2", "CPA3", "MS4A2", "KIT", "GATA2"], vmax = 100)

In [ ]:
relabel_cluster_celltypes(adata,new_label="Mast Cells ",target_clusters=["17"])

In [ ]:
sc.pl.umap(adata, color = ['cell_type'], frameon = False,title="LEIDEN CLUSTERING")

In [ ]:
sc.tl.rank_genes_groups(
    adata, groupby="leiden", method="wilcoxon", key_added="dea_leiden"
)

In [ ]:
sc.tl.dendrogram(adata, groupby='leiden')

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="leiden", standard_scale="var", n_genes=5, key="dea_leiden"
)


In [ ]:
# Extract the resuls of differential genes
ranked_genes = adata.uns['dea_leiden']

# Obtain the processed groups
groups = ranked_genes['names'].dtype.names

# Create a dictionary to store top 5 genes per group
top_genes_by_group = {}

# Iterate through each group and extract top 5 genes
for group in groups:
    top_genes_by_group[group] = ranked_genes['names'][group][:10]  # Tomar los primeros 5 genes

# Show results
for group, genes in top_genes_by_group.items():
    print(f"Grupo {group}: {', '.join(genes)}")


## Paper annotation

In [ ]:
# Only read columns "Barcodes" and "Celltypes" 
df = pd.read_csv("cell_annotation/GSE263733_Cell_annotation.txt", sep="\t", usecols=["Barcodes", "Celltypes"])

# Verify the first rows to confirm columns
#print(df.head()) 

# Create dictionary
diccionario = pd.Series(df.Celltypes.values, index=df.Barcodes).to_dict()

In [ ]:
print(dict(list(diccionario.items())[:10]))

In [ ]:
adata.obs['cell_type_2'] = adata.obs.index.map(diccionario)

In [ ]:
sc.pl.umap(adata, color = ['cell_type'] + ['cell_type_2'], frameon = False)

### CONCLUSIONES
I have noticed two discrepancies: in the B cell cluster (it seems that I labeled part of them as T cells), and one cluster that I believe belongs to mast cells.
The rest matches the annotation obtained from GEO.

In [ ]:
adata.write("adata_anotado.h5ad")

In [ ]:
import scanpy as sc

In [ ]:
adata = sc.read_h5ad('GSE263733_cleaned.h5ad')

In [ ]:
adata

In [ ]:
import pandas as pd

# Contar células por tipo celular
conteo_por_tipo = adata.obs['cell_type'].value_counts()

# Mostrar el resultado
print(conteo_por_tipo)